## Scrapping & Preprocessing

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import os

HEADERS = {
    "User-Agent": "Mozilla/5.0"
}

author_base_urls = [
    #"https://www.culture.ru/literature/poems/author-osip-mandelshtam",#------
    "https://www.culture.ru/literature/poems/author-anna-akhmatova",
    #"https://www.culture.ru/literature/poems/author-mikhail-lermontov",#-------
    #"https://www.culture.ru/literature/poems/author-aleksandr-pushkin", #потом
    #"https://www.culture.ru/literature/poems/author-sergei-esenin",
    #"https://www.culture.ru/literature/poems/author-vladimir-solovev",
    #"https://www.culture.ru/literature/poems/author-nikolai-zabolockii",
    #"https://www.culture.ru/literature/poems/author-vladimir-mayakovskii",
    #"https://www.culture.ru/literature/poems/author-aleksandr-blok",
    #"https://www.culture.ru/literature/poems/author-iosif-brodskii",
    #"https://www.culture.ru/literature/poems/author-fedor-tyutchev",
    #"https://www.culture.ru/literature/poems/author-marina-cvetaeva",
    #"https://www.culture.ru/literature/poems/author-afanasii-fet",
    #"https://www.culture.ru/literature/poems/author-robert-rozhdestvenskii",
    #"https://www.culture.ru/literature/poems/author-vladimir-vysockii",
    #"https://www.culture.ru/literature/poems/author-nikolai-gumilev",
    #"https://www.culture.ru/literature/poems/author-nikolai-nekrasov",
    #"https://www.culture.ru/literature/poems/author-boris-pasternak", #----
    #"https://www.culture.ru/literature/poems/author-ivan-bunin",
    #"https://www.culture.ru/literature/poems/author-anton-delvig",
    #"https://www.culture.ru/literature/poems/author-kondratiy-rileev",
    #"https://www.culture.ru/literature/poems/author-evgenii-boratynskii-baratynskii",
    #"https://www.culture.ru/literature/poems/author-vasilii-zhukovskii",
    #"https://www.culture.ru/literature/poems/author-vilgelm-kyukhelbeker",
    #"https://www.culture.ru/literature/poems/author-nikolai-gnedich",
    #"https://www.culture.ru/literature/poems/author-velimir-hlebnikov",
    #"https://www.culture.ru/literature/poems/author-andrei-belyi"
]

all_poems = []

def get_all_pages(base_url):
    """Возвращает список всех страниц стихов автора."""
    response = requests.get(base_url, headers=HEADERS)
    soup = BeautifulSoup(response.content, 'html.parser')

    page_links = soup.select('a.CsObS[href*="page="]')
    max_page = 1
    for link in page_links:
        href = link.get("href", "")
        if "page=" in href:
            try:
                page_num = int(href.split("page=")[-1])
                max_page = max(max_page, page_num)
            except ValueError:
                pass

    url_base = base_url.split("?page=")[0]
    return [f"{url_base}?page={i}" for i in range(1, max_page + 1)]

def get_poem_links(page_url):
    response = requests.get(page_url, headers=HEADERS)
    soup = BeautifulSoup(response.content, 'html.parser')
    cards = soup.find_all("div", class_="CHPy6")
    links = []
    for card in cards:
        a_tag = card.find("a", class_="ICocV")
        if a_tag and 'href' in a_tag.attrs:
            poem_url = "https://www.culture.ru" + a_tag['href']
            links.append(poem_url)
    return links

def parse_poem(poem_url):
    response = requests.get(poem_url, headers=HEADERS)
    soup = BeautifulSoup(response.content, 'html.parser')

    author_tag = soup.find("div", class_="HjkFX")
    author = author_tag.get_text(strip=True) if author_tag else "Unknown"

    title_tag = soup.find("div", class_="rrWFt")
    title = title_tag.get_text(strip=True) if title_tag else "Untitled"

    text_blocks = soup.find_all("div", attrs={"data-content": "text"})
    stanzas = []
    total_lines = 0

    for block in text_blocks:
        stanza_lines = []
        for elem in block.children:
            if isinstance(elem, str):
                text = elem.strip()
                if text:
                    stanza_lines.append(text)
            elif elem.name == "br":
                stanza_lines.append("\n")
            elif elem.name:
                text = elem.get_text(strip=True)
                if text:
                    stanza_lines.append(text)

        stanza_text = "".join(stanza_lines).strip()
        if stanza_text:
            stanzas.append(stanza_text)

    # Собираем полный текст
    text = "\n\n".join(stanzas)

    # Теперь аккуратно считаем строки:
    total_lines = text.count('\n') + 1

    return {
        "author": author,
        "title": title,
        "text": text,
        "count": total_lines,
        "url": poem_url
    }

# Главный цикл
for author_base_url in author_base_urls:
    print(f"Обрабатываем автора: {author_base_url}")
    author_pages = get_all_pages(author_base_url)
    for page_url in author_pages:
        print(f"  Страница: {page_url}")
        poem_links = get_poem_links(page_url)
        for poem_url in poem_links:
            try:
                print(f"    Стих: {poem_url}")
                poem_data = parse_poem(poem_url)
                all_poems.append(poem_data)
                time.sleep(1)
            except Exception as e:
                print(f"    Ошибка при обработке {poem_url}: {e}")

# Сохраняем результат
with open("akhmatova.json", "w", encoding="utf-8") as f:
    json.dump(all_poems, f, ensure_ascii=False, indent=2)

print("Сбор завершён. Стихотворения сохранены")

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import os

HEADERS = {
    "User-Agent": "Mozilla/5.0"
}

author_base_urls = [
    #"https://www.culture.ru/literature/poems/author-osip-mandelshtam",#------
    #"https://www.culture.ru/literature/poems/author-anna-akhmatova",
    #"https://www.culture.ru/literature/poems/author-mikhail-lermontov",#-------
    #"https://www.culture.ru/literature/poems/author-aleksandr-pushkin", #потом
    #"https://www.culture.ru/literature/poems/author-sergei-esenin",
    #"https://www.culture.ru/literature/poems/author-vladimir-solovev",
    #"https://www.culture.ru/literature/poems/author-vladimir-mayakovskii",
    #"https://www.culture.ru/literature/poems/author-aleksandr-blok",
    #"https://www.culture.ru/literature/poems/author-iosif-brodskii",
    #"https://www.culture.ru/literature/poems/author-fedor-tyutchev",
    #"https://www.culture.ru/literature/poems/author-marina-cvetaeva",
    #"https://www.culture.ru/literature/poems/author-afanasii-fet",
    #"https://www.culture.ru/literature/poems/author-robert-rozhdestvenskii",
    #"https://www.culture.ru/literature/poems/author-vladimir-vysockii",
    #"https://www.culture.ru/literature/poems/author-nikolai-gumilev",
    #"https://www.culture.ru/literature/poems/author-nikolai-nekrasov",
    #"https://www.culture.ru/literature/poems/author-boris-pasternak", #----
    #"https://www.culture.ru/literature/poems/author-ivan-bunin",
    #"https://www.culture.ru/literature/poems/author-anton-delvig",
    #"https://www.culture.ru/literature/poems/author-kondratiy-rileev",
    #"https://www.culture.ru/literature/poems/author-evgenii-boratynskii-baratynskii",
    #"https://www.culture.ru/literature/poems/author-vasilii-zhukovskii",
    #"https://www.culture.ru/literature/poems/author-vilgelm-kyukhelbeker",
    #"https://www.culture.ru/literature/poems/author-nikolai-gnedich",
    #"https://www.culture.ru/literature/poems/author-velimir-hlebnikov",
    #"https://www.culture.ru/literature/poems/author-andrei-belyi"
    "https://www.culture.ru/literature/poems/author-aleksandr-pushkin/tag-korotkie"
]

def get_all_pages(base_url):
    response = requests.get(base_url, headers=HEADERS)
    soup = BeautifulSoup(response.content, 'html.parser')

    page_links = soup.select('a.CsObS[href*="page="]')
    max_page = 1
    for link in page_links:
        href = link.get("href", "")
        if "page=" in href:
            try:
                page_num = int(href.split("page=")[-1])
                max_page = max(max_page, page_num)
            except ValueError:
                pass

    url_base = base_url.split("?page=")[0]
    return [f"{url_base}?page={i}" for i in range(1, max_page + 1)]

def get_poem_links(page_url):
    try:
        response = requests.get(page_url, headers=HEADERS)
        soup = BeautifulSoup(response.content, 'html.parser')
        cards = soup.find_all("div", class_="CHPy6")
        links = []
        for card in cards:
            a_tag = card.find("a", class_="ICocV")
            if a_tag and 'href' in a_tag.attrs:
                poem_url = "https://www.culture.ru" + a_tag['href']
                links.append(poem_url)
        return links
    except Exception as e:
        print(f"Ошибка при загрузке страницы {page_url}: {e}")
        return []

def parse_poem(poem_url):
    response = requests.get(poem_url, headers=HEADERS)
    soup = BeautifulSoup(response.content, 'html.parser')

    author_tag = soup.find("div", class_="HjkFX")
    author = author_tag.get_text(strip=True) if author_tag else "Unknown"

    title_tag = soup.find("div", class_="rrWFt")
    title = title_tag.get_text(strip=True) if title_tag else "Untitled"

    text_blocks = soup.find_all("div", attrs={"data-content": "text"})
    stanzas = []

    for block in text_blocks:
        stanza_lines = []
        for elem in block.children:
            if isinstance(elem, str):
                text = elem.strip()
                if text:
                    stanza_lines.append(text)
            elif elem.name == "br":
                stanza_lines.append("\n")
            elif elem.name:
                text = elem.get_text(strip=True)
                if text:
                    stanza_lines.append(text)
        stanza_text = "".join(stanza_lines).strip()
        if stanza_text:
            stanzas.append(stanza_text)

    text = "\n\n".join(stanzas)
    total_lines = text.count('\n') + 1

    return {
        "author": author,
        "title": title,
        "text": text,
        "count": total_lines,
        "url": poem_url
    }

def save_poems(poems, filename_base, part_number):
    filename = f"{filename_base}_part{part_number}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(poems, f, ensure_ascii=False, indent=2)
    print(f"Сохранено: {filename}")

# Главный цикл
for base_url in author_base_urls:
    raw_slug = base_url.split("/")[-1]  # например, "author-anna-akhmatova"
    filename_base = raw_slug.replace("author-", "")  # "anna-akhmatova"
    print(f"Обрабатываем: {filename_base}")

    all_pages = get_all_pages(base_url)
    page_chunks = [all_pages[i:i + 10] for i in range(0, len(all_pages), 10)]

    for i, chunk in enumerate(page_chunks, start=1):
        poems_chunk = []
        print(f"  Блок {i} (страницы {chunk[0]} ... {chunk[-1]})")

        for page_url in chunk:
            poem_links = get_poem_links(page_url)
            for poem_url in poem_links:
                try:
                    print(f"    Обработка: {poem_url}")
                    poem = parse_poem(poem_url)
                    poems_chunk.append(poem)
                    time.sleep(1)
                except Exception as e:
                    print(f"    Ошибка при разборе {poem_url}: {e}")

        if poems_chunk:
            save_poems(poems_chunk, filename_base, i)

print("Сбор завершён.")

In [ ]:
!pip install pillow

### json merging

In [ ]:
import json
import os

MAX_LENGTH = 14

# Папка, где лежат JSON-файлы с поэтами
folder_path = "separately"
output_file = "short_poems_corpus.json"

all_poems = []

# Проходим по каждому .json файлу в папке
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        print(f"📥 Обработка {file_path}")
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                poems = json.load(f)
                # Добавляем только короткие
                filtered = [poem for poem in poems if poem.get("count", 0) <= MAX_LENGTH]
                all_poems.extend(filtered)
        except Exception as e:
            print(f"Ошибка в {filename}: {e}")

# Сохраняем результат
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_poems, f, ensure_ascii=False, indent=2)

print(f"\n Готово. Сохранено {len(all_poems)} стихов в {output_file}")

## App

#### импорт

In [ ]:
import random
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
import json

#### test

In [ ]:
text = "I wandered lonely as a cloud\nThat floats on high o'er vales and hills"

# Размер экрана iPhone 13
width, height = 1170, 2532
background_color = (0, 0, 0)
text_color = (255, 255, 255)
font_size = 60

# Отступы по краям
margin = 10  # пикселей слева и справа

# Загружаем шрифт
font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", font_size)

img = Image.new("RGB", (width, height), background_color)
draw = ImageDraw.Draw(img)

# Готовим строки
lines = text.split('\n')
line_height = font.getbbox("A")[3] + 20
y = height // 2 - len(lines) * line_height // 2

for line in lines:
    line_width = font.getlength(line)
    # Теперь центрируем не по всей ширине, а в рамке между отступами
    max_text_width = width - 2 * margin
    x = margin + (max_text_width - line_width) // 2
    draw.text((x, y), line, font=font, fill=text_color)
    y += line_height

img.save("english_poem_with_margins2.jpg")

In [ ]:
# Параметры экрана iPhone 13
width, height = 1170, 2532
background_color = (0, 0, 0)
text_color = (255, 255, 255)
font_size = 40
margin = 2  # Увеличим отступы для красивого вида

# Загружаем шрифт с засечками, поддерживающий кириллицу
font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSerif.ttf"
font = ImageFont.truetype(font_path, font_size)

# Загружаем корпус
with open("poems2.json", "r", encoding="utf-8") as f:
    poems = json.load(f)

# Фильтруем стихотворения с 9 строками
poems_9_lines = [poem for poem in poems if poem["count"] == 14]

# Проверка: есть ли такие стихи
if not poems_9_lines:
    raise ValueError("Нет стихотворений с 9 строками в корпусе.")

# Выбираем случайное стихотворение
selected_poem = random.choice(poems_9_lines)
text = selected_poem["text"]

# Создаем изображение
img = Image.new("RGB", (width, height), background_color)
draw = ImageDraw.Draw(img)

# Подготовка строк
lines = text.split('\n')
line_height = font.getbbox("А")[3] + 20  # "А" для кириллицы
y = height // 2 - len(lines) * line_height // 2

# Отрисовка текста
for line in lines:
    line_width = font.getlength(line)
    max_text_width = width - 2 * margin
    x = margin + (max_text_width - line_width) // 2
    draw.text((x, y), line, font=font, fill=text_color)
    y += line_height

# Сохраняем
img.save("russian_poem_wallpaper1.jpg")

print(f"Готово! Использовано стихотворение: «{selected_poem['title']}» от {selected_poem['author']}")

In [ ]:
# Параметры
font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSerif.ttf"
font_size = 40
text_color = (255, 255, 255)
rectangle_color = (0, 0, 0, 180)  # полупрозрачный чёрный
margin = 50
line_spacing = 20

# Загружаем корпус
with open("short_poems_corpus.json", "r", encoding="utf-8") as f:
    poems = json.load(f)

# Фильтруем по длине (можно изменить число)
poems_filtered = [p for p in poems if p["count"] == 14]
if not poems_filtered:
    raise ValueError("Нет подходящих стихотворений.")

# Выбираем стихотворение
selected_poem = random.choice(poems_filtered)
lines = selected_poem["text"].split('\n')

# Загружаем фоновое изображение
background = Image.open("/Users/ilamusatkin/Diploma Code/Poetry/IMG_0635.JPG").convert("RGBA")
width, height = background.size

# Подготовка текста
font = ImageFont.truetype(font_path, font_size)
draw = ImageDraw.Draw(background)

# Вычисление размеров текста
line_heights = []
line_widths = []
for line in lines:
    bbox = font.getbbox(line)
    h = bbox[3] - bbox[1]
    w = font.getlength(line)
    line_heights.append(h)
    line_widths.append(w)

total_height = sum(line_heights) + line_spacing * (len(lines) - 1)
max_width = max(line_widths)

# Координаты прямоугольника
x1 = (width - max_width) // 2 - margin
y1 = (height - total_height) // 2 - margin
x2 = (width + max_width) // 2 + margin
y2 = (height + total_height) // 2 + margin

# Накладываем полупрозрачный прямоугольник
overlay = Image.new("RGBA", background.size, (0, 0, 0, 0))
overlay_draw = ImageDraw.Draw(overlay)
overlay_draw.rectangle([x1, y1, x2, y2], fill=rectangle_color)
background = Image.alpha_composite(background, overlay)

# Наносим текст
y_text = y1 + margin
for i, line in enumerate(lines):
    line_width = font.getlength(line)
    x_text = (width - line_width) // 2
    draw.text((x_text, y_text), line, font=font, fill=text_color)
    y_text += line_heights[i] + line_spacing

# Сохраняем
output = background.convert("RGB")
output.save("russian_poem_wallpaper2.jpg")

print(f"Готово! Стихотворение: «{selected_poem['title']}» от {selected_poem['author']}")

In [ ]:
!pip install --upgrade Pillow

In [ ]:
import json
import random
from PIL import Image, ImageDraw, ImageFont

# === параметры ===

CORPUS_PATH = "short_poems_corpus.json"
BACKGROUND_PATH = "/Users/ilamusatkin/Diploma Code/Poetry/IMG_0635.JPG"
OUTPUT_PATH = "russian_poem_wallpaper_final.jpg"
FONT_PATH = "/usr/share/fonts/truetype/dejavu/DejaVuSerif.ttf"

SCREEN_WIDTH = 1170
SCREEN_HEIGHT = 2532

FONT_SIZE = 40
LINE_SPACING = 20
MARGIN_TEXT = 50
MARGIN_TOP = 250     # отступ от верхнего края до названия
MARGIN_BOTTOM = 200  # отступ от нижнего края до автора

TEXT_COLOR = (255, 255, 255)
RECTANGLE_COLOR = (0, 0, 0, 180)

LINE_COUNT_FILTER = 14


# === загрузка корпуса ===

with open(CORPUS_PATH, "r", encoding="utf-8") as f:
    poems = json.load(f)

poems_filtered = [p for p in poems if p["count"] == LINE_COUNT_FILTER]
if not poems_filtered:
    raise ValueError("Нет подходящих стихотворений.")

selected_poem = random.choice(poems_filtered)
lines = selected_poem["text"].split('\n')
title = selected_poem["title"]
author = selected_poem["author"]


# === подготовка изображения ===

background = Image.open(BACKGROUND_PATH).convert("RGBA")
background = background.resize((SCREEN_WIDTH, SCREEN_HEIGHT))

font = ImageFont.truetype(FONT_PATH, FONT_SIZE)
italic_font = ImageFont.truetype(FONT_PATH, FONT_SIZE)

draw = ImageDraw.Draw(background)


# === подготовка текста ===

def get_text_size(text, font):
    bbox = font.getbbox(text)
    width = font.getlength(text)
    height = bbox[3] - bbox[1]
    return width, height


line_sizes = [get_text_size(line, font) for line in lines]
line_heights = [h for _, h in line_sizes]
total_height = sum(line_heights) + LINE_SPACING * (len(lines) - 1)

title_width, title_height = get_text_size(title, italic_font)
author_width, author_height = get_text_size(author, italic_font)

y_start = MARGIN_TOP + title_height + LINE_SPACING
y_end = SCREEN_HEIGHT - MARGIN_BOTTOM - author_height

available_height = y_end - y_start
if total_height > available_height:
    raise ValueError("Слишком длинное стихотворение для заданных отступов.")


# === затемняем фон в нужной области ===

rectangle_top = MARGIN_TOP
rectangle_bottom = SCREEN_HEIGHT - MARGIN_BOTTOM
rectangle_left = MARGIN_TEXT
rectangle_right = SCREEN_WIDTH - MARGIN_TEXT

overlay = Image.new("RGBA", background.size, (0, 0, 0, 0))
overlay_draw = ImageDraw.Draw(overlay)
overlay_draw.rectangle(
    [rectangle_left, rectangle_top, rectangle_right, rectangle_bottom],
    fill=RECTANGLE_COLOR
)
background = Image.alpha_composite(background, overlay)


# === наносим текст ===

draw = ImageDraw.Draw(background)

# название
x_title = (SCREEN_WIDTH - title_width) // 2
draw.text((x_title, MARGIN_TOP), title, font=italic_font, fill=TEXT_COLOR)

# стихотворение
y_text = y_start
for i, line in enumerate(lines):
    line_width, line_height = get_text_size(line, font)
    x_text = (SCREEN_WIDTH - line_width) // 2
    draw.text((x_text, y_text), line, font=font, fill=TEXT_COLOR)
    y_text += line_height + LINE_SPACING

# автор
x_author = SCREEN_WIDTH - MARGIN_TEXT - author_width
y_author = SCREEN_HEIGHT - MARGIN_BOTTOM
draw.text((x_author, y_author), author, font=italic_font, fill=TEXT_COLOR)


# === сохранение ===

output = background.convert("RGB")
output.save(OUTPUT_PATH)

print(f"Готово! Стихотворение: «{title}» от {author}")

### 28.05

In [ ]:
!pip install ipywidgets pillow matplotlib

In [ ]:
import json
import random
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np



# Настройки по умолчанию
DEFAULT_SETTINGS = {
    "CORPUS_PATH": "short_poems_corpus.json",
    "FONT_PATH": "/usr/share/fonts/truetype/dejavu/DejaVuSerif.ttf",
    "SCREEN_WIDTH": 1170,
    "SCREEN_HEIGHT": 2532,
    "FONT_SIZE": 40,
    "LINE_SPACING": 20,
    "MARGIN_TEXT": 50,
    "MARGIN_TOP": 250,
    "MARGIN_BOTTOM": 200,
    "TEXT_COLOR": (255, 255, 255),
    "RECTANGLE_COLOR": (0, 0, 0, 180),
    "LINE_COUNT_FILTER": 14
}

# Загрузка корпуса
def load_poems():
    try:
        with open(DEFAULT_SETTINGS["CORPUS_PATH"], "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception as e:
        print(f"Ошибка загрузки корпуса: {e}")
        return []

poems = load_poems()

# Функция генерации обоев
def generate_wallpaper(poem, bg_image=None, settings=DEFAULT_SETTINGS):
    if bg_image is None:
        bg_image = Image.new("RGB", (settings["SCREEN_WIDTH"], settings["SCREEN_HEIGHT"]), "black")
    else:
        bg_image = bg_image.resize((settings["SCREEN_WIDTH"], settings["SCREEN_HEIGHT"]))
    
    lines = poem["text"].split('\n')
    title = poem["title"]
    author = poem["author"]
    
    try:
        font = ImageFont.truetype(settings["FONT_PATH"], settings["FONT_SIZE"])
        italic_font = ImageFont.truetype(settings["FONT_PATH"], settings["FONT_SIZE"])
    except:
        font = ImageFont.load_default()
        italic_font = ImageFont.load_default()
    
    draw = ImageDraw.Draw(bg_image)
    
    def get_text_size(text, font):
        bbox = font.getbbox(text)
        return font.getlength(text), bbox[3] - bbox[1]
    
    line_sizes = [get_text_size(line, font) for line in lines]
    total_height = sum(h for _, h in line_sizes) + settings["LINE_SPACING"] * (len(lines) - 1)
    
    title_size = get_text_size(title, italic_font)
    author_size = get_text_size(author, italic_font)
    
    y_start = settings["MARGIN_TOP"] + title_size[1] + settings["LINE_SPACING"]
    y_end = settings["SCREEN_HEIGHT"] - settings["MARGIN_BOTTOM"] - author_size[1]
    
    overlay = Image.new("RGBA", bg_image.size, (0, 0, 0, 0))
    overlay_draw = ImageDraw.Draw(overlay)
    overlay_draw.rectangle(
        [settings["MARGIN_TEXT"], settings["MARGIN_TOP"],
         settings["SCREEN_WIDTH"] - settings["MARGIN_TEXT"], 
         settings["SCREEN_HEIGHT"] - settings["MARGIN_BOTTOM"]],
        fill=settings["RECTANGLE_COLOR"]
    )
    bg_image = Image.alpha_composite(bg_image, overlay)
    
    draw = ImageDraw.Draw(bg_image)
    x_title = (settings["SCREEN_WIDTH"] - title_size[0]) // 2
    draw.text((x_title, settings["MARGIN_TOP"]), title, font=italic_font, fill=settings["TEXT_COLOR"])
    
    y_text = y_start
    for line in lines:
        line_width, line_height = get_text_size(line, font)
        x_text = (settings["SCREEN_WIDTH"] - line_width) // 2
        draw.text((x_text, y_text), line, font=font, fill=settings["TEXT_COLOR"])
        y_text += line_height + settings["LINE_SPACING"]
    
    x_author = settings["SCREEN_WIDTH"] - settings["MARGIN_TEXT"] - author_size[0]
    y_author = settings["SCREEN_HEIGHT"] - settings["MARGIN_BOTTOM"]
    draw.text((x_author, y_author), author, font=italic_font, fill=settings["TEXT_COLOR"])
    
    return bg_image.convert("RGB")

# Создаем интерфейс
output = widgets.Output()
poem_selector = widgets.Dropdown(options=[p['title'] for p in poems], description='Стихотворение:')
author_selector = widgets.Dropdown(options=sorted(list(set(p['author'] for p in poems))), description='Автор:')
generate_btn = widgets.Button(description="Сгенерировать обои")
random_btn = widgets.Button(description="Случайное стихотворение")

font_size_slider = widgets.IntSlider(value=40, min=20, max=80, description='Размер шрифта:')
text_color_picker = widgets.ColorPicker(description='Цвет текста:', value='white')
opacity_slider = widgets.IntSlider(value=180, min=0,max=255, description='Прозрачность подложки:')

# Обработчики событий
def on_generate_click(b):
    with output:
        clear_output()
        selected_poem = next(p for p in poems if p['title'] == poem_selector.value)
        settings = DEFAULT_SETTINGS.copy()
        settings.update({
            "FONT_SIZE": font_size_slider.value,
            "TEXT_COLOR": tuple(int(text_color_picker.value.lstrip('#')[i:i+2], 16) for i in (0, 2, 4)),
            "RECTANGLE_COLOR": (0, 0, 0, opacity_slider.value)
        })
        
        img = generate_wallpaper(selected_poem, settings=settings)
        plt.figure(figsize=(10, 20))
        plt.imshow(np.array(img))
        plt.axis('off')
        plt.show()
        
        buf = BytesIO()
        img.save(buf, format="JPEG", quality=95)
        display(widgets.HTML(
            f'<a download="{selected_poem["title"]}.jpg" href="data:image/jpeg;base64,{buf.getvalue().hex()}" target="_blank">'
            'Скачать обои</a>'
        ))

def on_random_click(b):
    poem = random.choice(poems)
    poem_selector.value = poem['title']
    author_selector.value = poem['author']
    on_generate_click(b)

def on_author_change(change):
    if change['name'] == 'value':
        author_poems = [p for p in poems if p['author'] == change['new']]
        poem_selector.options = [p['title'] for p in author_poems]

generate_btn.on_click(on_generate_click)
random_btn.on_click(on_random_click)
author_selector.observe(on_author_change)

# Отображаем интерфейс
display(widgets.VBox([
    widgets.HBox([random_btn]),
    widgets.HBox([author_selector, poem_selector]),
    font_size_slider,
    text_color_picker,
    opacity_slider,
    generate_btn,
    output
]))